<a href="https://colab.research.google.com/github/JeromeNickson/Quantum-Portfolio-Optimization/blob/main/notebooks/qunatum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies (run once)
!pip install pennylane yfinance cvxpy --quiet

# Imports
import numpy as np
import pandas as pd
import yfinance as yf
import cvxpy as cp
import pennylane as qml
from pennylane import numpy as pnp
import matplotlib.pyplot as plt
import math
from collections import Counter

# ---------------------------
# 1) Data: same tickers/timeframe as classical
# ---------------------------
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "META"]
start = "2020-01-01"
end   = "2022-01-01"

prices = yf.download(tickers, start=start, end=end)["Close"]
returns = prices.pct_change().dropna()

# Annualize (same convention as classical)
mu = returns.mean().values * 252        # annual expected returns (vector shape (n,))
Sigma = returns.cov().values * 252      # annual covariance matrix (n x n)
n = len(tickers)

# ---------------------------
# 2) Classical tangency (for comparison) - compute via efficient frontier scanning
# ---------------------------
def solve_markowitz(mu, Sigma, target_return):
    n = len(mu)
    w = cp.Variable(n)
    port_ret = mu @ w
    port_var = cp.quad_form(w, Sigma)
    prob = cp.Problem(cp.Minimize(port_var),
                      [cp.sum(w) == 1, w >= 0, port_ret >= target_return])
    prob.solve(solver=cp.SCS, verbose=False)  # SCS is robust; change if you prefer
    if prob.status == "optimal":
        return w.value, float(port_ret.value), float(port_var.value)
    return None, None, None

min_r = float(np.min(mu))
max_r = float(np.max(mu))
target_returns = np.linspace(min_r, max_r, 40)

risks, rets, weights = [], [], []
for r in target_returns:
    w_opt, pret, pvar = solve_markowitz(mu, Sigma, r)
    if w_opt is not None:
        risks.append(np.sqrt(pvar))
        rets.append(pret)
        weights.append(w_opt)

# compute tangency (max Sharpe) from frontier
rf = 0.02
sharpe_ratios = [(r - rf)/s if s>0 else -1 for r,s in zip(rets, risks)]
if len(sharpe_ratios) == 0:
    raise RuntimeError("No feasible classical portfolios found — adjust targets/dates.")
max_idx = int(np.argmax(sharpe_ratios))
classical_weights = weights[max_idx]
classical_return = rets[max_idx]
classical_risk = risks[max_idx]
classical_sharpe = sharpe_ratios[max_idx]

print("Classical tangency (Markowitz) results:")
for t,w in zip(tickers, classical_weights):
    print(f"  {t}: {w:.4f}")
print(f"  Return: {classical_return:.4f}, Risk: {classical_risk:.4f}, Sharpe: {classical_sharpe:.4f}\n")

# ---------------------------
# 3) Build QUBO for cardinality selection
#    Variables x_i in {0,1}; choose exactly k assets.
#    Objective: lam * x^T Sigma x - (1-lam) * mu^T x + A*(sum(x)-k)^2
# ---------------------------
k = 3               # choose exactly k assets (tuneable)
lam = 0.6           # risk vs return trade-off (0..1)
A = 10.0            # penalty weight for cardinality constraint (tuneable)

Q = np.zeros((n,n))
# Risk term (quadratic)
Q += lam * Sigma
# Return (linear) -> minimize negative return
for i in range(n):
    Q[i,i] += - (1.0 - lam) * mu[i]
# Cardinality penalty
for i in range(n):
    Q[i,i] += A * (1 - 2*k)   # diag: A*(1 - 2k)
for i in range(n):
    for j in range(i+1, n):
        Q[i,j] += A           # off-diagonal: A
        Q[j,i] += A

# Q is now the QUBO matrix such that cost = x^T Q x

# ---------------------------
# 4) Convert QUBO -> Ising (z in {-1,+1}) such that cost = const + sum h_i z_i + sum J_ij z_i z_j
#    Using mapping x = (1 - z)/2.
# ---------------------------
def qubo_to_ising(Q):
    n = Q.shape[0]
    const = 0.0
    h = np.zeros(n)
    J = np.zeros((n,n))
    # Using algebra:
    # x_i = (1 - z_i)/2
    # x^T Q x => const + sum h_i z_i + sum_{i<j} J_ij z_i z_j
    # Derivation yields:
    # const = 0.5 * sum_i Q_ii + 0.5 * sum_{i<j} Q_ij
    # h_i = -0.5*Q_ii - 0.5*sum_{j != i} Q_ij
    # J_ij = 0.5 * Q_ij  (for i != j)
    const = 0.5 * np.sum(np.diag(Q)) + 0.5 * np.sum(np.triu(Q,1))
    for i in range(n):
        h[i] = -0.5 * Q[i,i] - 0.5 * np.sum(Q[i,:]) + 0.5 * Q[i,i]  # careful remove double-counted diag
        # simplify: -0.5*Q_ii -0.5*sum_j Q_ij + 0.5*Q_ii = -0.5 * sum_{j != i} Q_ij
        h[i] = -0.5 * np.sum(Q[i,:]) + 0.0 * Q[i,i]
    for i in range(n):
        for j in range(i+1, n):
            J[i,j] = 0.5 * Q[i,j]
            J[j,i] = J[i,j]
    return const, h, J

const, h, J = qubo_to_ising(Q)

# Build PennyLane Hamiltonian H_cost = const + sum_i h_i Z_i + sum_{i<j} J_ij Z_i Z_j
coeffs = []
ops = []
for i in range(n):
    if abs(h[i]) > 1e-12:
        coeffs.append(float(h[i]))
        ops.append(qml.PauliZ(i))
for i in range(n):
    for j in range(i+1, n):
        if abs(J[i,j]) > 1e-12:
            coeffs.append(float(J[i,j]))
            ops.append(qml.PauliZ(i) @ qml.PauliZ(j))
H_cost = qml.Hamiltonian(coeffs, ops)

# ---------------------------
# 5) QAOA ansatz and variational minimization (expectation value)
# ---------------------------
p = 2  # QAOA depth (tune)
dev = qml.device("default.qubit", wires=n)

@qml.qnode(dev, interface="autograd")
def qaoa_expectation(gammas, betas):
    # initial |+> state
    for i in range(n):
        qml.Hadamard(wires=i)
    # alternating cost and mixer layers
    for layer in range(p):
        qml.ApproxTimeEvolution(H_cost, gammas[layer], 1)   # cost unitary approx e^{-i gamma H}
        for i in range(n):
            qml.RX(2 * betas[layer], wires=i)               # mixer
    return qml.expval(H_cost)

# Initialize parameters (pennylane numpy so gradients work)
rng = np.random.default_rng(42)
gammas = pnp.array(rng.uniform(0, 2*np.pi, size=(p,)), requires_grad=True)
betas  = pnp.array(rng.uniform(0, 2*np.pi, size=(p,)), requires_grad=True)

# Optimizer
opt = qml.AdamOptimizer(stepsize=0.1)
max_iters = 60   # adjust to trade-off runtime vs quality
print("Running QAOA optimization (this may take a minute)...")
for it in range(max_iters):
    gammas, betas, cost_val = opt.step_and_cost(lambda g,b: qaoa_expectation(g,b), gammas, betas)
    if (it+1) % 10 == 0 or it==0:
        print(f"  Iter {it+1:3d}: expectation = {cost_val:.6f}")

print("Optimization done. Final expectation:", qaoa_expectation(gammas, betas))

# ---------------------------
# 6) Sampling from optimized QAOA circuit (shot-based) to get bitstrings
# ---------------------------
shots = 2000
dev_sample = qml.device("default.qubit", wires=n, shots=shots)

@qml.qnode(dev_sample)
def qaoa_sample(gammas, betas):
    for i in range(n):
        qml.Hadamard(wires=i)
    for layer in range(p):
        qml.ApproxTimeEvolution(H_cost, float(gammas[layer]), 1)
        for i in range(n):
            qml.RX(2 * float(betas[layer]), wires=i)
    return qml.sample(wires=range(n))   # returns array (shots, n) of bits 0/1

samples = qaoa_sample(gammas, betas)  # shape (shots, n)
# Convert to (shots, n) numpy array
samples = np.array(samples)

# Count top few bitstrings
bitstrings = ["".join(map(str, s)) for s in samples]
counts = Counter(bitstrings)
top = counts.most_common(8)
print("\nTop measurement outcomes (bitstring : count):")
for bs, c in top:
    print(f"  {bs} : {c}")

# Evaluate QUBO cost for each unique bitstring and find best satisfying cardinality
unique_bs = list(counts.keys())
best_cost = 1e18
best_x = None
for bs in unique_bs:
    x = np.array(list(map(int, list(bs))))
    cost_val = x @ Q @ x
    # prefer exact cardinality k; penalize otherwise
    if np.sum(x) == k:
        if cost_val < best_cost:
            best_cost = cost_val
            best_x = x

# If nothing with exact k found, pick the lowest cost overall
if best_x is None:
    for bs in unique_bs:
        x = np.array(list(map(int, list(bs))))
        cost_val = x @ Q @ x + 100.0 * (np.sum(x) - k)**2
        if cost_val < best_cost:
            best_cost = cost_val
            best_x = x

print("\nBest selected bitstring (x):", best_x, " cost:", best_cost)

# ---------------------------
# 7) Convert selection to equal-weighted portfolio and evaluate metrics
# ---------------------------
if np.sum(best_x) == 0:
    # fallback: choose top-k by mu
    idx = np.argsort(-mu)[:k]
    best_x = np.zeros(n, dtype=int)
    best_x[idx] = 1

w_qaoa = best_x / np.sum(best_x)   # equal-weighted
ret_q = float(w_qaoa @ mu)
risk_q = float(np.sqrt(w_qaoa @ Sigma @ w_qaoa))
sharpe_q = (ret_q - rf) / risk_q if risk_q>0 else np.nan

print("\nQAOA-selected portfolio (equal weights):")
for t,xi in zip(tickers, best_x):
    print(f"  {t}: {xi}")
print(f"  Return: {ret_q:.4f}, Risk: {risk_q:.4f}, Sharpe: {sharpe_q:.4f}")

# ---------------------------
# 8) Quick comparison table
# ---------------------------
print("\n--- Comparison ---")
print(f"Classical (Markowitz Tangency): Return={classical_return:.4f}, Risk={classical_risk:.4f}, Sharpe={classical_sharpe:.4f}")
print(f"Quantum (QAOA selection k={k}):        Return={ret_q:.4f}, Risk={risk_q:.4f}, Sharpe={sharpe_q:.4f}")
